# Generate IDV_teleport script.

In [1]:
import pandas as pd #only requirement

In [2]:
raw=pd.read_csv('Igel_WEIO_case_list.txt',sep='\s+',names=['syyyy','smm','sdd','eyyyy','emm','edd'])

In [3]:
raw.head() #s prefix for start and e prefix for end

,syyyy,smm,sdd,eyyyy,emm,edd
0,1997,2,27,1997,3,4
1,1997,2,28,1997,3,5
2,1997,9,30,1997,10,3
3,1999,2,21,1999,2,27
4,1999,4,22,1999,4,27


### Teleport script needs a centered date and time delta to teleport, so calculate center of the state and timedelta 

In [4]:
startdate=raw.apply(lambda x:pd.datetime(x['syyyy'],x['smm'],x['sdd']),axis=1)
enddate=raw.apply(lambda x:pd.datetime(x['eyyyy'],x['emm'],x['edd']),axis=1)+pd.to_timedelta(1,unit='days')

In [5]:
raw['startdate']=startdate
raw['timedelta']=((enddate-startdate)/2.0).apply(lambda x:pd.to_timedelta(x.days,unit='days'))

In [6]:
raw['teleportdate']=raw['startdate']+raw['timedelta']

In [7]:
raw.head()

,syyyy,smm,sdd,eyyyy,emm,edd,startdate,timedelta,teleportdate
0,1997,2,27,1997,3,4,1997-02-27,3 days,1997-03-02
1,1997,2,28,1997,3,5,1997-02-28,3 days,1997-03-03
2,1997,9,30,1997,10,3,1997-09-30,2 days,1997-10-02
3,1999,2,21,1999,2,27,1999-02-21,3 days,1999-02-24
4,1999,4,22,1999,4,27,1999-04-22,3 days,1999-04-25


### Generate teleport script by applying a function to every row of the dataframe and save it

In [8]:
executable="idv_teleport"
bundle="-b ERAI_MERRA2_TPW_2soundings_IRsat.xidv"
bbox='-bbox 17 36 -15 90' #NW SW
extra="-nohead True"
#needed -t -td -case
def teleport_str(x):
    tdate=str(x['teleportdate']).split(' ')[0]
    td=str(x['timedelta'].days)+'days'
    case='Igel_WEIO_case_'+str(x['startdate']).split(' ')[0]
    return ' '.join([executable,bundle,'-t',tdate,'-td',td,bbox,'-case',case,extra])

In [9]:
teleport_str=raw.apply(teleport_str,axis=1)

In [10]:
teleport_str[0]

'idv_teleport -b ERAI_MERRA2_TPW_2soundings_IRsat.xidv -t 1997-03-02 -td 3days -bbox 17 36 -15 90 -case Igel_WEIO_case_1997-02-27 -nohead True'

In [11]:
teleport_str.to_csv('teleport_Igel_WEIO_cases.sh',sep='\n',index=False)